In [ ]:
from cosmikyu import gan, config, datasets, transforms, utils
from cosmikyu import nn as cnn
import os
import torch
import mlflow
from orphics import maps
import numpy as np

data_dir = config.default_data_dir
sehgal_dir = os.path.join(data_dir, 'sehgal')
cuda = True
compts = ["kappa", "ksz", "tsz", "ir_pts", "rad_pts"]
compt_idxes = [0, 1, 2, 3, 4]
shape = (10, 128, 128)
sample_interval = 500
save_interval = 1
batch_size = 32
nepochs = 100
norm_info_file = "/home/dwhan89/workspace/cosmikyu/data/sehgal/201020_logzshrink_normalization_info_validation.npz"
_, wcs = maps.rect_geometry(width_arcmin=64., px_res_arcmin=0.5)

# Configure data loader
os.makedirs(data_dir, exist_ok=True)
os.makedirs(sehgal_dir, exist_ok=True)
SDN = transforms.SehgalDataNormalizerScaledLogZShrink(norm_info_file)
RF = transforms.RandomFlips(p_v=0.5, p_h=0.5)
SDS_target = datasets.SehgalDataSet(sehgal_dir, "train141020", transforms=[SDN, RF], dummy_label=False, dtype=np.float64)
SDS_input = datasets.SehgalDataSet(sehgal_dir, "train_tertiary081120", transforms=[SDN], dummy_label=False, dtype=np.float32)
DSJ = datasets.DataSetJoiner([SDS_input,SDS_target],  dtype=np.float64, shape=(10, 128, 128), dummy_label=True)

dataloader = torch.utils.data.DataLoader(
    DSJ,
    batch_size=batch_size,
    shuffle=True,
)


LF = cnn.LinearFeature(5,5, bias=True)
experiment_id = "06b9a352b8bb4051b50f91660ebc4cfe"
model_dir = "/home/dwhan89/workspace/cosmikyu/output/sehgal_pixganwgp_301020/{}/model".format(experiment_id)
FORSE = gan.FORSE_WGP("sehgal_forse_081020", shape, nconv_fcgen=64,
                          nconv_fcdis=64, cuda=False, ngpu=4, nconv_layer_gen=4, nconv_layer_disc=4, kernal_size=4, stride=2,
                          padding=1, output_padding=0, gen_act=[LF], nin_channel=5, nout_channel=5, nthresh_layer_gen=0, nthresh_layer_disc=0, dropout_rate=0)
#FORSE.load_states(model_dir, "_{}".format(13))
mlflow.set_experiment(FORSE.identifier)
with mlflow.start_run(experiment_id=FORSE.experiment.experiment_id) as mlflow_run:
    torch.cuda.empty_cache()
    FORSE.train(
        dataloader,
        nepochs=nepochs,
        ncritics=5,
        sample_interval=sample_interval,
        save_interval=save_interval,
        load_states=True,
        save_states=True,
        verbose=True,
        mlflow_run=mlflow_run,
        lr=1e-4,
        betas=(0.5, 0.9),
        lambda_gp=10.
    )



/home/dwhan89/.miniconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
Traceback (most recent call last):
  File "/home/dwhan89/.miniconda3/lib/python3.7/site-packages/mlflow/store/tracking/file_store.py", line 197, in list_experiments
    experiment = self._get_experiment(exp_id, view_type)
  File "/home/dwhan89/.miniconda3/lib/python3.7/site-packages/mlflow/store/tracking/file_store.py", line 260, in _get_experiment
    meta = read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
  File "/home/dwhan89/.miniconda3/lib/python3.7/site-packages/mlflow/utils/file_utils.py", line 167, in read_yaml
    raise MissingConfigException("Yaml file '%s' does not exist." % file_path)
mlflow.exceptions.M

Number of joined samples are 200000
[WARNING] You have a CUDA device. You probably want to run with CUDA enabled


Traceback (most recent call last):
  File "/home/dwhan89/.miniconda3/lib/python3.7/site-packages/mlflow/store/tracking/file_store.py", line 197, in list_experiments
    experiment = self._get_experiment(exp_id, view_type)
  File "/home/dwhan89/.miniconda3/lib/python3.7/site-packages/mlflow/store/tracking/file_store.py", line 260, in _get_experiment
    meta = read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
  File "/home/dwhan89/.miniconda3/lib/python3.7/site-packages/mlflow/utils/file_utils.py", line 167, in read_yaml
    raise MissingConfigException("Yaml file '%s' does not exist." % file_path)
mlflow.exceptions.MissingConfigException: Yaml file '/home/dwhan89/workspace/cosmikyu/cosmikyu/../output/mlruns/mlruns/meta.yaml' does not exist.


loading saved states 
failed to load saved states
saving states _0
[Epoch 0/100] [Batch 0/6250] [D loss: 9.466755] [G loss: 0.189648]
[Epoch 0/100] [Batch 5/6250] [D loss: 8.027654] [G loss: 1.181412]
[Epoch 0/100] [Batch 10/6250] [D loss: 4.324178] [G loss: 3.631386]
[Epoch 0/100] [Batch 15/6250] [D loss: -6.993964] [G loss: 9.824821]
